In [37]:
### Creating Combat Log Table

In [38]:
# what needs to happen is that a table needs to be created that stores each turn in combat and the actions that happened in that turn.
# the table should have the following columns:

# create the table in the form of a dataframe
import pandas as pd
import numpy as np
import sqlite3
    
# create the table
combat_log = pd.DataFrame(columns=[
                                   'Combat Round',
                                   'Action Number',
                                   'Action Time', # action, bonus action, reaction, etc
                                   'Action Name', # attack, cast spell, dodge, etc
                                   'Action Type', # attack, support, heal, control, etc
                                   'Target',
                                   'Current Allied Ability Check',
                                   'Current Allied Attack Roll',
                                   'Current Allied Saving Throw',
                                   'Current Allied Damage Roll',
                                   'Current Enemy Ability Check',
                                   'Current Enemy Attack Roll',
                                   'Current Enemy Saving Throw',
                                   'Current Enemy Damage Roll',
                                   ])

In [39]:
import Species
import Backgrounds
import Armor_and_Weapons
import Dice_Rolls
import Establishing_Hierarchy
import CHARACTER_CREATOR
import Monsters

import Character_Actions

In [40]:
def generate_situation(character_num):
    # for the number of characters, randomly generate a Player_Character using Random_Character()
    entities = []
    for i in range(character_num):
        entities.append(CHARACTER_CREATOR.Random_Character())

    # for the number of monsters, ask which monsters to add 

    # for the number of entities, within each entity's flesh out the Active_Conditions and Circumstances dictionaries
    for entity in range(len(entities)):
        other_entities = entities.copy()
        other_entities.pop(entity)
        
        # I wnant to add a key for each other entity in the Active_Conditions dictionaries
        for other_entity in range(len(other_entities)):
            entities[entity].Active_Conditions[str(entities[other_entity].Name)] = []

    return entities

def generate_combat(character_num):
    # create a new combat log
    combat_log_new = pd.DataFrame(columns=[
                                   'Combat Round',
                                   'Action Number',
                                   'Action Time', # action, bonus action, reaction, etc
                                   'Action Name', # attack, cast spell, dodge, etc
                                   'Action Type', # attack, support, heal, control, etc
                                   'Target',
                                   'Current Allied Ability Check',
                                   'Current Allied Attack Roll',
                                   'Current Allied Saving Throw',
                                   'Current Allied Damage Roll',
                                   'Current Enemy Ability Check',
                                   'Current Enemy Attack Roll',
                                   'Current Enemy Saving Throw',
                                   'Current Enemy Damage Roll',
                                   ])
    # create situation
    combat_situation = generate_situation(character_num)
    
    # add the Character_Actions.Attack_Action(entity,combat_situation) to each entity's actions
    for entity in range(len(combat_situation)):
        combat_situation[entity].Actions['Attack'] = Character_Actions.Attack_Action(combat_situation[entity],combat_situation,combat_log_new)

    for entity in range(len(combat_situation)):
        Character_Actions.Equip_Weapon(combat_situation[entity],Character_Actions.Choose_Random_Weapon(combat_situation[entity]))

    # alter the combat log to include the details of every entity in the situation
    # loop through each creature in the situation and add a column for each attribute of the creature
    for creature in range(len(combat_situation)):
        creature_reference = combat_situation[creature]
        combat_log_new[str(creature_reference.Name) + ' Acting True'] = 0
        
        combat_log_new[str(creature_reference.Name) + ' Size'] = creature_reference.Size
        combat_log_new[str(creature_reference.Name) + ' Walking Speed'] = creature_reference.Speed['Walking']
        combat_log_new[str(creature_reference.Name) + ' Flying Speed'] = creature_reference.Speed['Flying']
        combat_log_new[str(creature_reference.Name) + ' Str_Score'] = creature_reference.Str_Score
        combat_log_new[str(creature_reference.Name) + ' Dex_Score'] = creature_reference.Dex_Score
        combat_log_new[str(creature_reference.Name) + ' Con_Score'] = creature_reference.Con_Score
        combat_log_new[str(creature_reference.Name) + ' Int_Score'] = creature_reference.Int_Score
        combat_log_new[str(creature_reference.Name) + ' Wis_Score'] = creature_reference.Wis_Score
        combat_log_new[str(creature_reference.Name) + ' Cha_Score'] = creature_reference.Cha_Score
        combat_log_new[str(creature_reference.Name) + ' Current_HP'] = creature_reference.Current_HP
        combat_log_new[str(creature_reference.Name) + ' Temp_HP'] = creature_reference.Temp_HP
        #combat_log[str(creature_reference.Name) + 'Weapon_Equipped'] = creature_reference.Weapon_Equipped
        #combat_log[str(creature_reference.Name) + 'Armor_Equipped'] = creature_reference.Armor_Equipped
        combat_log_new[str(creature_reference.Name) + ' Active_Conditions'] = creature_reference.Active_Conditions['Self']
        combat_log_new[str(creature_reference.Name) + ' Concentrating'] = creature_reference.Concentrating



    # loop that continues combat until conclusion (aka when one creature's current hp is 0 or less)
        # loop that records each round in 
    combat_round = 0
    combat_log_new['Combat Round'] = combat_round
    #while True:
    #    combat_round += 1
    #    combat_log_new['Combat Round'] = combat_round

    # use a for loop to have each entity take the Attack_Action
    for entity in range(len(combat_situation)):
        new_round = combat_situation[entity].Actions['Attack']
        # concat the dict new_round to the combat_log_new
        combat_log_new = pd.concat([combat_log_new, pd.DataFrame(new_round)], ignore_index=False)


    return combat_log_new


In [41]:
# remove the number of columns limit
pd.set_option('display.max_columns', None)

In [42]:
generate_combat(4)

Character_Creator - Random Character Function - Class Choice:  <module 'Barbarian' from 'c:\\Users\\maxhi\\OneDrive\\Documents\\GitHub\\5e-Simulation\\Barbarian.py'>
Character_Creator - Random Character Function - Class Choice:  <module 'Bard' from 'c:\\Users\\maxhi\\OneDrive\\Documents\\GitHub\\5e-Simulation\\Bard.py'>
Character_Creator - Random Character Function - Class Choice:  <module 'Artificer' from 'c:\\Users\\maxhi\\OneDrive\\Documents\\GitHub\\5e-Simulation\\Artificer.py'>
Character_Creator - Random Character Function - Class Choice:  <module 'Artificer' from 'c:\\Users\\maxhi\\OneDrive\\Documents\\GitHub\\5e-Simulation\\Artificer.py'>
0
0
0
0


ValueError: If using all scalar values, you must pass an index